In [1]:
!pip install -q ragas rapidfuzz
!pip install -q sentence_transformers tiktoken lark
!pip install -q langchain langchain-core langchain-google-genai


[notice] A new release of pip is available: 23.2.1 -> 25.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 25.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


# Fitness evaluation

In [1]:
from dotenv import load_dotenv
import os
import pandas as pd
import time

load_dotenv()

gemini_api_key = os.getenv("GEMINI_API_KEY")
openrouter_api_key = os.getenv("OPENROUTER_API_KEY")
avalai_api_key = os.getenv("AVALAI_API_KEY")

In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.rate_limiters import InMemoryRateLimiter

# rate_limiter = InMemoryRateLimiter(requests_per_second=0.1, check_every_n_seconds=10, max_bucket_size=10)

# qwen = ChatOpenAI(
#     model="qwen/qwen-2.5-72b-instruct:free",
#     temperature=0,
#     api_key=openrouter_api_key,
#     base_url="https://openrouter.ai/api/v1",
# #     rate_limiter=rate_limiter,
# )

AVALAI_BASE_URL = "https://api.avalai.ir/v1/"
MODEL = "gpt-4.1-mini"

llm = ChatOpenAI(
    model=MODEL,
    base_url=AVALAI_BASE_URL,
    api_key=os.environ["AVALAI_API_KEY"],
    temperature=0,
)

ragas_llm = ChatOpenAI(
    model="gpt-4.1-mini",
    base_url=AVALAI_BASE_URL,
    api_key=os.environ["AVALAI_API_KEY"],
    temperature=0,
)


In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.rate_limiters import InMemoryRateLimiter

rate_limiter = InMemoryRateLimiter(requests_per_second=0.1, check_every_n_seconds=10, max_bucket_size=10)
MODEL_NAME = "gemini-2.5-flash-preview-05-20"

gemini_chat = ChatGoogleGenerativeAI(model=MODEL_NAME, temperature=0, api_key=gemini_api_key, rate_limiter=rate_limiter)
# gemini_chat_half = ChatGoogleGenerativeAI(model=MODEL_NAME, temperature=0.5, api_key=gemini_api_key, rate_limiter=rate_limiter)
# gemini_chat_1 = ChatGoogleGenerativeAI(model=MODEL_NAME, temperature=1, api_key=gemini_api_key, rate_limiter=rate_limiter)


/tmp/ipykernel_77904/1010678895.py:4: LangChainBetaWarning: Introduced in 0.2.24. API subject to change.
  rate_limiter = InMemoryRateLimiter(requests_per_second=0.1, check_every_n_seconds=10, max_bucket_size=10)


In [4]:
from ragas.llms import LangchainLLMWrapper
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en-v1.5")
ragas_llm = LangchainLLMWrapper(ragas_llm)

/tmp/ipykernel_77904/1055483862.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en-v1.5")


In [5]:
import pandas as pd

records_df = pd.read_csv('./data/evaluation/fitness/records.csv')

In [6]:
from db.db_manager import FitnessDB

db_manager = FitnessDB(db_path='./fitness_temp1.db', use_dummy_data=False)

SQLite database './fitness_temp1.db' created/verified with updated schema.


In [7]:
from werkzeug.security import generate_password_hash

for i, user in records_df.iterrows():
    db_manager.create_user(
        email = user['email'],
        password_hash = generate_password_hash(user['password']),
        first_name = user['first_name'],
        last_name = user['last_name'],
        date_of_birth = user['date_of_birth'],
        sex =  user['sex'],
        height_cm = user['height_cm'],
        weight_kg = user['weight_kg'],
        activity_level = user['activity_level'],
        fitness_goals = user['fitness_goals'],
    )

In [8]:
from agents.planner import PlannerGraph
from langchain_core.messages import HumanMessage
from db.retrievers import DocumentRetriever
from agents.enums import PlannerType

agent = PlannerGraph(
    llm = llm,
    db_manager = db_manager,
    book_retriever = DocumentRetriever('data/'),
    planner_type = PlannerType.FITNESS,
    num_results = 5,
    use_rag_data = True,
    summarize_logs = False # disable log summarization.
).compile()

Making the book retriever ready =====
table is already created=====
Book Retriever is ready ====


In [9]:
from IPython.display import clear_output
from tqdm import tqdm

results = {
    "answers": [],
    "contexts": []
}

for i, user in tqdm(list(records_df.iterrows())):
    response = agent.invoke({"messages": HumanMessage(content=user['query']), 'user_id':i+1}, config={"configurable": {"thread_id": i+1}})
    results["answers"].append(response["messages"][-1].content)
    results["contexts"].append(response["contexts"])
    clear_output(wait=True)


100%|██████████| 37/37 [06:07<00:00,  9.92s/it]


In [10]:
from datasets import Dataset

response_dataset = Dataset.from_dict({
    "question" : records_df['query'].tolist(),
    "answer" : results["answers"],
    "contexts" : results["contexts"],
})


In [11]:
response_dataset.save_to_disk('./data/evaluation/fitness/ragas/ragas_dataset_tables')


Saving the dataset (0/1 shards):   0%|          | 0/37 [00:00<?, ? examples/s]

In [12]:
from datasets import load_from_disk
response_dataset = load_from_disk('./data/evaluation/fitness/ragas/ragas_dataset_tables')


In [13]:
from ragas import evaluate
from ragas.metrics import faithfulness, answer_relevancy

metrics = [
    faithfulness,
    answer_relevancy,
]

In [14]:
from tqdm import tqdm

per_sample = []
for i in tqdm(range(response_dataset.num_rows)):
    result = evaluate(
        dataset=response_dataset.select([i]),
        metrics=metrics,
        llm=ragas_llm,
        embeddings=embeddings,
    )
    per_sample.append(result.scores[0])
    print(f"{i+1}: {result.scores[0]}")
#     time.sleep(10)


  0%|          | 0/37 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/2 [00:00<?, ?it/s]

  3%|▎         | 1/37 [00:24<14:58, 24.97s/it]

1: {'faithfulness': 0.8333333333333334, 'answer_relevancy': 0.7642431543446232}


Evaluating:   0%|          | 0/2 [00:00<?, ?it/s]

...

 97%|█████████▋| 36/37 [31:29<01:02, 62.46s/it]

36: {'faithfulness': 0.04081632653061224, 'answer_relevancy': 0.771953995459596}


Evaluating:   0%|          | 0/2 [00:00<?, ?it/s]

100%|██████████| 37/37 [32:31<00:00, 52.74s/it]

37: {'faithfulness': 0.10526315789473684, 'answer_relevancy': 0.7529407421336924}


In [15]:
import pandas as pd

pd.DataFrame(per_sample).to_csv('./data/evaluation/fitness/ragas/ragas_eval_tables.csv', index=False)

In [16]:
import pandas as pd

ragas_eval = pd.read_csv("./data/evaluation/fitness/ragas/ragas_eval_tables.csv")

In [17]:
ragas_eval.mean()

faithfulness        0.171086
answer_relevancy    0.772432
dtype: float64

# Diet evaluation

In [20]:
import pandas as pd

records_df = pd.read_csv('./data/evaluation/diet/diet_records.csv')

In [21]:
from datasets import Dataset
import ast

response_dataset = Dataset.from_dict({
    "question" : records_df['query'].tolist(),
    "answer" :records_df['agent_plan'].tolist(),
    "contexts" : [ast.literal_eval(x) for x in records_df['agent_context']],
})


In [ ]:
response_dataset.save_to_disk('./data/evaluation/diet/ragas/diet_ragas_dataset')


In [ ]:
from datasets import load_from_disk
response_dataset = load_from_disk('./data/evaluation/diet/ragas/diet_ragas_dataset')


In [ ]:
from ragas import evaluate
from ragas.metrics import faithfulness, answer_relevancy

metrics = [
    faithfulness,
    answer_relevancy,
]

In [ ]:
from tqdm import tqdm

per_sample = []
for i in tqdm(range(response_dataset.num_rows)):
    result = evaluate(
        dataset=response_dataset.select([i]),
        metrics=metrics,
        llm=ragas_llm,
        embeddings=embeddings,
    )
    per_sample.append(result.scores[0])
    print(f"{i+1}: {result.scores[0]}")
#     time.sleep(10)


  0%|          | 0/40 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/2 [00:00<?, ?it/s]

  2%|▎         | 1/40 [00:38<24:44, 38.07s/it]

1: {'faithfulness': 0.03571428571428571, 'answer_relevancy': 0.7713490563020953}


Evaluating:   0%|          | 0/2 [00:00<?, ?it/s]

...

 98%|█████████▊| 39/40 [27:03<00:39, 39.26s/it]

39: {'faithfulness': 0.15217391304347827, 'answer_relevancy': 0.710098252834571}


Evaluating:   0%|          | 0/2 [00:00<?, ?it/s]

100%|██████████| 40/40 [27:41<00:00, 41.53s/it]

40: {'faithfulness': 0.0, 'answer_relevancy': 0.7939876370677256}


In [ ]:
import pandas as pd

pd.DataFrame(per_sample).to_csv('./data/evaluation/diet/ragas/diet_ragas_eval.csv', index=False)

In [24]:
import pandas as pd

ragas_eval = pd.read_csv("./data/evaluation/diet/ragas/diet_ragas_eval.csv")

In [25]:
ragas_eval.mean()

faithfulness        0.034015
answer_relevancy    0.695350
dtype: float64